In [ ]:
# EXAMPLES OF PATH FILES TO ENTER IN THE INTERFACE :
# COMMENT : DO NOT RUN THIS STEP 
C:/Users/user_name/Documents/ArcGIS/Projects/name_ArcGIS_project
C:/Users/user_name/Documents/file_for_data/DATA_SEISMIC_RISKS

In [8]:
# COMMENT : RUN THIS PART LAST

# LIBRARIES
import arcpy
from tkinter import *
from tkinter import ttk

# EXTENSION
arcpy.CheckOutExtension("Spatial")

# INSERTION OF ALL THE VARIABLES NEEDED FOR THE PROJECT
variables = WindowDataInsertion()

# WORSPACE ENVIRONMENT
root = str(variables[2]) + "/"
name_outgdb = str(variables[3]) + ".gdb"
data_path = str(variables[4])
arcpy.env.workspace = root + name_outgdb

# MAIN PROGRAM 
# Creation of the output file for the analysis and the data needed
Initialisation(root, name_outgdb)
InsertData(data_path, root, name_outgdb)

arcpy.env.workspace = root + name_outgdb
# When running the code, an AttributeError can happen, such as : 'ToolValidator' object has 
# no attribute 'isLicensed'. It is a ArcGIS Pro problem, the program will continue to run anyway. 

# Create and fill the new columns needed for the volcanoes data
SetParameters(root + name_outgdb)

# Take into account the map choice, "Human risks" or "Economic risks" to determine the function needed
choice = str(variables[1])

if choice =="Human Risks" :
    # Create all the intermediate rater files needed and the final raster file for the analysis
    Final_Population_Risks_Analysis(root, name_outgdb, data_path, variables)
else:
    # Create all the intermediate rater files needed and the final raster file for the analysis
    Final_Economic_Risks_Analysis(root, name_outgdb, data_path, variables)
print("Final results ready, to see the analysis results go to ArcGIS Pro.")
print("This program was carried out as part of the Geospatial Programming course, collaboratively developed by RUDN and EPF.")

Creating C:/Users/mletaill/Documents/ArcGIS/Projects/MyProjectV1/out_gdb34.gdb


AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

Major earthquakes layer selection created.
Kernel selection created.
IDW layer created.
Final Earthquakes Analysis completed.
Volcano analysis completed.
Final results ready, to see the analysis results go to ArcGIS Pro.
This program was carried out as part of the Geospatial Programming course, collaboratively developed by RUDN and EPF.


In [6]:
# COMMENT : RUN THIS PART OF THE NOTEBOOK FIRST TO INITIALIZE ALL THE FUNCTIONS NEEDED

# NAME: Initialisation 
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Create the output environment for the analysis
def Initialisation(root, name_outgdb):
    
    # Allow to overwrite
    arcpy.env.overwriteOutput = True
    
    # Creation of the gdb file from ArcPy 
    outgdb = root + name_outgdb 
    
    if arcpy.Exists(root + outgdb):
        arcpy.Delete_management(outgdb)

    if not arcpy.Exists(root + outgdb):
        print("Creating", outgdb)
        arcpy.CreateFileGDB_management(root, name_outgdb)

# NAME: InsertData 
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Insert the data needed for the analysis in the output file in order to be accessible for the analysis 
def InsertData(data_workspace, root, outgdb):
    # Select the path where the data are stored
    arcpy.env.workspace = data_workspace
    
    # Insert in the output gdb file all the files needed for the analysis 
    arcpy.RasterToGeodatabase_conversion("Population_Density_2020.tif", root + outgdb)
    arcpy.RasterToGeodatabase_conversion("Urban.tif", root + outgdb)
    arcpy.FeatureClassToGeodatabase_conversion("SignificantVolcanicEruption.shp", root + outgdb)
    arcpy.FeatureClassToGeodatabase_conversion("TectonicPlate.shp", root + outgdb)

# NAME: SetParameters
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Create new columns in the different tables
def SetParameters(workspace):
    arcpy.env.workspace = workspace
    
    # Addition of the column necessary for the analysis in the volcanoes table
    arcpy.AddField_management("SignificantVolcanicEruption","Next_Eruption","FLOAT")
    arcpy.AddField_management("SignificantVolcanicEruption","Distance_VEI","TEXT")
    arcpy.AddField_management("TectonicPlate","Presence","FLOAT")

    # Calculation of the distance depending on the VEI and the next possible eruption
    DistanceInsertion("SignificantVolcanicEruption", "Distance_VEI")
    NextEruption("SignificantVolcanicEruption","Next_Eruption")
    arcpy.CalculateField_management("TectonicPlate","Presence","100")

# NAME: DistanceInsertion
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Calculate and insert the distance depending on the VEI of every volcano in the table 
def DistanceInsertion(name_table, name_column):
    expression = "getDistanceVEI(!VEI!)"
    codeblock = """
def getDistanceVEI(VEI):
        if VEI == 1:
            return "1 Kilometers"
        elif VEI == 2:
            return "5 Kilometers"
        elif VEI == 3:
            return "15 Kilometers"
        elif VEI == 4:
            return "25 Kilometers"
        elif VEI == 5:
            return "35 Kilometers"
        elif VEI == 6:
            return "40 Kilometers"
        elif VEI == 7:
            return "50 Kilometers"
    """
    # Calculation of the column 
    arcpy.CalculateField_management(name_table,name_column,expression,"PYTHON3",codeblock)

# NAME: NextEruption
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Calculate and insert the next eruption depending on the VEI of every volcano in the table 
def NextEruption(name_table,name_column):
    expression = "getNextEruption(!Year!,!VEI!)"
    codeblock = """
def getNextEruption(Year, VEI):
        if VEI == 1:
            return (2020 - Year - 1/365)
        if VEI == 2:
            return (2020 - Year - 1/52)
        if VEI == 3:
            return (2020 - Year - 1)
        if VEI == 4:
            return (2020 - Year - 10)
        if VEI == 5:
            return (2020 - Year - 100)
        if VEI == 6:
            return (2020 - Year - 100)
        if VEI == 7:
            return (2020 - Year - 1000)
    """
    # Calculation of the column
    arcpy.CalculateField_management(name_table,name_column,expression,"PYTHON3",codeblock)

# NAME: NextEruption
# ARCGIS EXTENSION NEEDED: Spatial Analyst
# PURPOSE: Create the different buffers and rasters for the analysis  
def BufferAndRasterCreation(root, name_outgdb, in_buffer_distance):
    arcpy.env.workspace = root + name_outgdb

    # Create a buffer around the volcano depending on the volcano VEI 
    arcpy.Buffer_analysis("SignificantVolcanicEruption", "Buffer_Table_Volcanoes", "Distance_VEI")

    # Create a Raster Layer depending on either the next eruption or the elevation of the volcano
    arcpy.PolygonToRaster_conversion("Buffer_Table_Volcanoes","Elevation","Raster_Volcano_Elevation", "CELL_CENTER", "", 10000)
    arcpy.PolygonToRaster_conversion("Buffer_Table_Volcanoes","Next_Eruption","Raster_Volcano_Next_Eruption", "CELL_CENTER", "", 10000)

    # Create a buffer around the tectonic plates depending on the user input
    arcpy.Buffer_analysis("TectonicPlate", "Buffer_File_Tectonic_Plates", in_buffer_distance)

    # Create a raster layer in order to be analyze with the other layers 
    arcpy.PolygonToRaster_conversion("Buffer_File_Tectonic_Plates","Presence","Raster_Tectonic_Plates")

# NAME: MajorEarthquakeLayerSelection
# ARCGIS EXTENSION NEEDED: Spatial Analyst
# PURPOSE: Create the selection for the kernel density of major earthquake
def MajorEarthquakeLayerSelection(datestart,dateend):
    # Find the file in all the feature classes
    feacla = arcpy.ListFeatureClasses()
    for fc in feacla:
        if fc == "SelectionByIntensityMajorEarthquankes.shp":
            str1 = fc
    # Creation of a selection if it doesn't exist
    if not arcpy.Exists("selection_lyr"):
        arcpy.MakeFeatureLayer_management(str1, "selection_lyr")
    
    # Create the query and layer from the user input dates
    qry = "YEAR >= "+ str(datestart) + " And YEAR < " + str(dateend)
    selectionName = "SelectionMajorEarthquakesFrom" + str(datestart) + "to" + str(dateend)
    arcpy.SelectLayerByAttribute_management("selection_lyr", "ADD_TO_SELECTION", qry)
    
    if arcpy.Exists(selectionName + ".shp"):
        arcpy.Delete_management(selectionName + ".shp")
    
    arcpy.CopyFeatures_management("selection_lyr", selectionName + ".shp")
    print("Major earthquakes layer selection created.")
    return selectionName


# NAME: CreatingKernelDensity
# ARCGIS EXTENSION NEEDED: Spatial Analyst
# PURPOSE: Create the Kernel density of a specify selection of major earthquakes and rescale it for later treatement
def CreatingKernelDensity(datestart,dateend, root, output):
    arcpy.env.overwriteOutput = True
    layerName = MajorEarthquakeLayerSelection(datestart,dateend)
    
    # Create a kernel from the input dates of the user
    KernelName = "KernelDensityForMajorEarthquakesFrom" + str(datestart) + "to" + str(dateend)
    OutKer = arcpy.sa.KernelDensity(layerName + ".shp", "NONE")
    
    # Rescale the function from 0 to 100 to match with the other functions
    OutResKer = arcpy.sa.RescaleByFunction(OutKer, arcpy.sa.TfMSLarge (1, 1), 0, 100)
    OutResKer.save(root + output + "/Kernel_Rescale")
    
    print("Kernel selection created.")
    return OutResKer

# NAME: CreateRecentEarthquakes
# ARCGIS EXTENSION NEEDED: Spatial Analyst
# PURPOSE: Create the IDW for the recent earthquakes layer 
def CreateRecentEarthquakes(root, output):
    arcpy.env.overwriteOutput = True
    data = "RecentEarthquakesMagnitude.shp"
   
    # Creation of the layer utilizing the IDW function depending on the magnitude 
    OutRecIDW = arcpy.sa.Idw(data, "mag")
   
    OutResRecIDW = arcpy.sa.RescaleByFunction(OutRecIDW, arcpy.sa.TfMSLarge (1, 1), 0, 100)
    OutResRecIDW.save(root + output + "/" + "RecentEarthquakesIDW" + "Rescale")
    
    print("IDW layer created.")
    return OutResRecIDW

# NAME: FinalEarthquakesAnalysis
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Create the final raster adding the different earthquakes layers
def FinalEarthquakesAnalysis(root, name_outgdb, start, end, workspace, coeff_Kernel, coeff_IDW):
    arcpy.env.workspace = workspace
    
    # Retreive of the Kernel and IDW for the earthquakes analysis
    OutResKer = CreatingKernelDensity(start,end, root, name_outgdb)
    OutResRecIDW = CreateRecentEarthquakes(root, name_outgdb)
    
    # Creation of the final raster for the earthquakes layer with the coefficient 
    arcpy.env.workspace = root + name_outgdb
    Raster_Final_Earthquakes_Analysis = OutResRecIDW*coeff_IDW + OutResKer*coeff_Kernel
    Raster_Final_Earthquakes_Analysis.save(root + name_outgdb + "/Raster_Final_Earthquakes_Analysis")
    
    print("Final Earthquakes Analysis completed.")
    return Raster_Final_Earthquakes_Analysis
    
# NAME: FinalVolcanoesAnalysis
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Create the final raster adding the different volcanoes layers
def FinalVolcanoesAnalysis(root, name_outgdb, coeff_Eruption, coeff_Elevation, in_buffer_distance):
    arcpy.env.workspace = root + name_outgdb

    # Create the 2 buffers needed and the Rasters for the analysis
    BufferAndRasterCreation(root, name_outgdb, in_buffer_distance)
    
    # Rescale the 2 raster volcanoes layers 
    Norm_Volcano_Eruption = arcpy.sa.RescaleByFunction("Raster_Volcano_Next_Eruption", "",0,100)
    Norm_Volcano_Elevation = arcpy.sa.RescaleByFunction("Raster_Volcano_Elevation", "",0,100)
    
    # Add the 2 volcanoes layers according to the coefficients chosen by the user 
    Raster_Add_Volcano_Analysis = Norm_Volcano_Eruption*coeff_Eruption + Norm_Volcano_Elevation*coeff_Elevation + 1
    Raster_Add_Volcano_Analysis.save(root + name_outgdb + "/Raster_Add_Volcano_Analysis")
    
    # Create a final raster layer with 0 where there is "No Data", so that the earthquakes and volcanoes layers can be added
    Raster_Final_Volcano_Analysis = arcpy.sa.Con(arcpy.sa.IsNull("Raster_Add_Volcano_Analysis"),0,"Raster_Add_Volcano_Analysis")
    Raster_Final_Volcano_Analysis.save(root + name_outgdb + "/Raster_Final_Volcano_Analysis")
    
    print("Volcano analysis completed.")
    return Raster_Final_Volcano_Analysis

# NAME: Final_Population_Risks_Analysis
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Add all the layers together according to the coefficients inserted by the user 
def Final_Population_Risks_Analysis(root, name_outgdb, workspace, variables):
    # Taking all the variables inserted by the user
    earth_start = str(variables[5])
    earth_end = str(variables[6])
 
    kernel_coeff = float(variables[7]) 
    IDW_coeff = float(variables[8]) 
 
    buffer_distance = str(variables[9]) + " " + str(variables[0])
    in_buffer_distance = str(buffer_distance)
 
    factorV_1 = float(variables[10])
    factorV_2 = float(variables[11])
 
    factorG_1 = float(variables[12])
    factorG_2 = float(variables[13])
    factorG_3 = float(variables[14])
    factorG_4 = float(variables[15])
    factorG_5 = float(variables[16])
    factorG_6 = float(variables[17])

    # Call all the functions created above for the analysis
    Raster_Final_Earthquakes_Analysis = FinalEarthquakesAnalysis(root, name_outgdb, earth_start, earth_end, workspace, kernel_coeff, IDW_coeff)
    Raster_Final_Volcano_Analysis = FinalVolcanoesAnalysis(root, name_outgdb, factorV_1, factorV_2, in_buffer_distance)
    
    # Create and rescale all the data layers needed for the analysis (Human risks only)
    Final_Seismic_Risks = Raster_Final_Earthquakes_Analysis*factorG_1 + Raster_Final_Volcano_Analysis*factorG_2
    Final_Seismic_Risks.save(root + name_outgdb + "/Final_Seismic_Risks")
    Norm_Seismic_Risks = arcpy.sa.RescaleByFunction("Final_Seismic_Risks", "",0,100)
    Norm_Seismic_Risks.save(root + name_outgdb + "/Norm_Seismic_Risks")
    
    Norm_Population_Density = arcpy.sa.RescaleByFunction("Population_Density_2020", "",0,100)
    Norm_Population_Density.save(root + name_outgdb + "/Norm_Population_Density")
    
    # Add all the layers together with the coefficients 
    Raster_Final_Population_Risks = Norm_Seismic_Risks*factorG_6 + Norm_Population_Density*factorG_3 + arcpy.Raster("Raster_Tectonic_Plates")*factorG_5
    Raster_Final_Population_Risks.save(root + name_outgdb + "/Raster_Final_Population_Risks")

# NAME: Final_Economic_Risks_Analysis
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Add all the layers together according to the coefficients inserted by the user 
def Final_Economic_Risks_Analysis(root, name_outgdb, workspace, variables):
    # Taking all the variables inserted by the user
    earth_start = str(variables[5])
    earth_end = str(variables[6])
 
    kernel_coeff = float(variables[7]) 
    IDW_coeff = float(variables[8]) 
 
    buffer_distance = str(variables[9]) + " " + str(variables[0])
    in_buffer_distance = str(buffer_distance)
 
    factorV_1 = float(variables[10])
    factorV_2 = float(variables[11])
 
    factorG_1 = float(variables[12])
    factorG_2 = float(variables[13])
    factorG_3 = float(variables[14])
    factorG_4 = float(variables[15])
    factorG_5 = float(variables[16])
    factorG_6 = float(variables[17])

    # Call all the functions created above for the analysis
    Raster_Final_Earthquakes_Analysis = FinalEarthquakesAnalysis(root, name_outgdb, earth_start, earth_end, workspace, kernel_coeff, IDW_coeff)
    Raster_Final_Volcano_Analysis = FinalVolcanoesAnalysis(root, name_outgdb, factorV_1, factorV_2, in_buffer_distance)
    
    # Create and rescale all the data layers needed for the analysis (Human risks only)
    Final_Seismic_Risks = Raster_Final_Earthquakes_Analysis*factorG_1 + Raster_Final_Volcano_Analysis*factorG_2
    Final_Seismic_Risks.save(root + name_outgdb + "/Final_Seismic_Risks")
    Norm_Seismic_Risks = arcpy.sa.RescaleByFunction("Final_Seismic_Risks", "",0,100)
    Norm_Seismic_Risks.save(root + name_outgdb + "/Norm_Seismic_Risks")
    
    Norm_Population_Density = arcpy.sa.RescaleByFunction("Population_Density_2020", "",0,100)
    Norm_Population_Density.save(root + name_outgdb + "/Norm_Population_Density")
    
    # Add all the layers together with the coefficients 
    Raster_Final_Economic_Risks = Norm_Seismic_Risks*factorG_6 + Norm_Population_Density*factorG_3 + arcpy.Raster("Urban")*factorG_3 +arcpy.Raster("Raster_Tectonic_Plates")*factorG_5
    Raster_Final_Economic_Risks.save(root + name_outgdb + "/Raster_Final_Economic_Risks")

In [7]:
# COMMENT : RUN THIS PART OF THE NOTEBOOK BEFORE THE MAIN PROGRAM TO INITIALIZE THE WINDOW 

# NAME: WindowDataInsertion 
# ARCGIS EXTENSION NEEDED: none
# PURPOSE: Create the window were the user insert all the parameters
def WindowDataInsertion():
    
    # Creation of th list that will contain all the variables needed for the analysis 
    variables = []
    total = 0
    
    # Creation of the window
    window = Tk()
 
    # Creation of all the frames in order to have one side with the name and the other to write 
    frame0 = Frame(window, width=768, height=576, borderwidth=1)
    frame0.pack(fill=BOTH)
    
    frame1 = Frame(window, width=768, height=576, borderwidth=1)
    frame1.pack(fill=BOTH)
    
    frame2 = Frame(window, width=768, height=576, borderwidth=1)
    frame2.pack(fill=BOTH)
    
    frame3 = Frame(window, width=768, height=576, borderwidth=1)
    frame3.pack(fill=BOTH)
    
    frame0bis = Frame(window, width=768, height=576, borderwidth=1)
    frame0bis.pack(fill=BOTH)
    
    frame4 = Frame(window, width=768, height=576, borderwidth=1)
    frame4.pack(fill=BOTH)
    
    frame5 = Frame(window, width=768, height=576, borderwidth=1)
    frame5.pack(fill=BOTH)
    
    frame0ter = Frame(window, width=768, height=576, borderwidth=1)
    frame0ter.pack(fill=BOTH)
    
    frame6 = Frame(window, width=768, height=576, borderwidth=1)
    frame6.pack(fill=BOTH)
    
    frame7 = Frame(window, width=768, height=576, borderwidth=1)
    frame7.pack(fill=BOTH)
    
    frame0qua = Frame(window, width=768, height=576, borderwidth=1)
    frame0qua.pack(fill=BOTH)
 
    frame8 = Frame(window, width=768, height=576, borderwidth=1)
    frame8.pack(fill=BOTH)
    
    frame9 = Frame(window, width=768, height=576, borderwidth=1)
    frame9.pack(fill=BOTH)
 
    frame10 = Frame(window, width=768, height=576, borderwidth=1)
    frame10.pack(fill=BOTH)
    
    frame11 = Frame(window, width=768, height=576, borderwidth=1)
    frame11.pack(fill=BOTH)
    
    # Creation of all the label and entry needed for the analysis 
    title_label = Label(frame0, text="MAPPING THE HUMAN AND ECONOMIC RISKS \nLINKED TO THE SEISMIC ACTIVITY IN THE WORLD", fg="red")
    title_label.pack()
    
    root_label = Label(frame1, text="PATH FILES : ")
    root_label.pack(side = LEFT)
 
    # Provide the variable to receive the text entered
    root_value=StringVar()  
    root=Entry(frame1, textvariable=root_value,width=55) 
    root.pack(side = RIGHT)
 
    output_label = Label(frame2, text="OUTPUT FILE NAME : ")
    output_label.pack(side = LEFT)
 
    # For the output file name
    output_value=StringVar()  
    output=Entry(frame2, textvariable=output_value,width=55) 
    output.pack(side = RIGHT) 
    
    data_label = Label(frame3, text="DATA FOLDER PATH : ")
    data_label.pack(side = LEFT)
 
    # For the data folder path
    data_value=StringVar()  
    data_folder=Entry(frame3, textvariable=data_value,width=55) 
    data_folder.pack(side = RIGHT)
    
    
    earthQ_label = Label(frame0bis, text="EARTHQUAKES DATA", fg = "red")
    earthQ_label.pack()
    
    start_label = Label(frame4, text="EARTHQUAKES START DATE : ")
    start_label.pack(side = LEFT)
    start_value=StringVar()  
    earth_start=Entry(frame4, textvariable=start_value,width=7) 
    earth_start.pack(side = LEFT)
    
    end_label = Label(frame4, text="END DATE : ")
    end_label.pack(side = LEFT)
    end_value=StringVar()  
    earth_end=Entry(frame4, textvariable=end_value,width=7) 
    earth_end.pack(side = LEFT)
    
    kernel_label = Label(frame5, text="KERNEL COEFFICIENT : ")
    kernel_label.pack(side = LEFT)
    kernel_value=StringVar()  
    kernel_entry=Entry(frame5, textvariable=kernel_value,width=7) 
    kernel_entry.pack(side = LEFT)
    
    IDW_label = Label(frame5, text="IDW COEFFICIENT : ")
    IDW_label.pack(side = LEFT)
    IDW_value=StringVar()  
    IDW_entry=Entry(frame5, textvariable=IDW_value,width=7) 
    IDW_entry.pack(side = LEFT)
    
    
    volca_label = Label(frame0ter, text="VOLCANOES DATA", fg="red")
    volca_label.pack()
    
 
    buffer_label = Label(frame6, text="BUFFER DISTANCE : ")
    buffer_label.pack(side = LEFT)
    buffer_value=StringVar()  
    buffer_distance=Entry(frame6, textvariable=buffer_value,width=7) 
    buffer_distance.pack(side = LEFT)
    
    def action(event):
        # Obtain the element selected
        select = listMetric.get()
        variables.append(select)
    
    metric = ["Meters","Kilometers", "Miles"]
    listMetric = ttk.Combobox(frame6, values = metric)
    listMetric.pack(side = LEFT)
    listMetric.bind("<<ComboboxSelected>>", action)
    
        
    factorV_1_label = Label(frame7, text="VOLCANO ELEVATION RISK : ")
    factorV_1_label.pack(side = LEFT)
    factorV_1_value=StringVar()  
    factorV_1=Entry(frame7, textvariable=factorV_1_value,width=7) 
    factorV_1.pack(side = LEFT) 
    
    factorV_2_label = Label(frame7, text="VOLCANO ERUPTION RISK : ")
    factorV_2_label.pack(side = LEFT)
    factorV_2_value=StringVar()  
    factorV_2=Entry(frame7, textvariable=factorV_2_value,width=7) 
    factorV_2.pack(side = LEFT) 
    
    global_label = Label(frame0qua, text="GLOBAL DATA", fg="red")
    global_label.pack()
    
    factorG_1_label = Label(frame8, text="EARTHQUAKES GLOBAL : ")
    factorG_1_label.pack(side = LEFT)
    factorG_1_value=StringVar()  
    factorG_1=Entry(frame8, textvariable=factorG_1_value,width=7) 
    factorG_1.pack(side = LEFT)    
    
    factorG_2_label = Label(frame8, text="VOLCANOES GLOBAL :    ")
    factorG_2_label.pack(side = LEFT)
    factorG_2_value=StringVar()  
    factorG_2=Entry(frame8, textvariable=factorG_2_value,width=7) 
    factorG_2.pack(side = LEFT) 
    
        
    factorG_3_label = Label(frame9, text="POPULATION DENSITY :   ")
    factorG_3_label.pack(side = LEFT)
    factorG_3_value=StringVar()  
    factorG_3=Entry(frame9, textvariable=factorG_3_value,width=7) 
    factorG_3.pack(side = LEFT)    
    
    factorG_4_label = Label(frame9, text="INFRASTRUCTURES : ")
    factorG_4_label.pack(side = LEFT)
    factorG_4_value=StringVar()  
    factorG_4=Entry(frame9, textvariable=factorG_4_value,width=7) 
    factorG_4.pack(side = LEFT) 
    
    factorG_5_label = Label(frame10, text="TECTONIC PLATES :         ")
    factorG_5_label.pack(side = LEFT)
    factorG_5_value=StringVar()  
    factorG_5=Entry(frame10, textvariable=factorG_5_value,width=7) 
    factorG_5.pack(side = LEFT)
    
    factorG_6_label = Label(frame10, text="SEISMIC ACTIVITY :         ")
    factorG_6_label.pack(side = LEFT)
    factorG_6_value=StringVar()  
    factorG_6=Entry(frame10, textvariable=factorG_6_value,width=7) 
    factorG_6.pack(side = LEFT)
    
    Layer_label = Label(frame11, text="WHICH MAP DO YOU WANT : ")
    Layer_label.pack(side = LEFT)
    def action2(event):
        # Obtenir l'élément sélectionné
        select = listLayers.get()
        variables.append(select)
        #print(select)
 
    select_Layer = ["Human Risks","Economic Risks"]
    listLayers = ttk.Combobox(frame11, values = select_Layer)
    listLayers.pack(side = LEFT)
    listLayers.bind("<<ComboboxSelected>>", action2)
 
    window.mainloop()
        
    # Store the variable in a list in order to be returned 
    variables.append(root_value.get())
    variables.append(output_value.get())
    variables.append(data_value.get())
    
    variables.append(start_value.get())
    variables.append(end_value.get())
    
    variables.append(kernel_value.get())
    variables.append(IDW_value.get())
 
    variables.append(buffer_value.get())
    
    
    variables.append(factorV_1_value.get())
    variables.append(factorV_2_value.get())   
    
    variables.append(factorG_1_value.get())
    variables.append(factorG_2_value.get())
    variables.append(factorG_3_value.get())
    variables.append(factorG_4_value.get())
    variables.append(factorG_5_value.get())
    variables.append(factorG_6_value.get())
 
    return variables